# We need to dump data for face genders and identities

In [ ]:
import json
import os

from esper.spark_util import *

STORIES_DIR = '/app/data/stories-data/'

if not os.path.exists(STORIES_DIR):
    os.makedirs(STORIES_DIR)

IDENTS_PATH = os.path.join(STORIES_DIR, 'identities_by_video.json')
GENDERS_AND_RACE_PATH = os.path.join(STORIES_DIR, 'genders_race_by_video.json')

In [ ]:
# gender_race_data = {
#     video_id: [
#         (gender_id, start_time, is_black, is_host)
#     ]
# }

# identity_data = {
#     video_id: [
#         (ident_id, start_time)
#     ]
# }

## Race data

In [ ]:
BLACK_SCORE_THRESHOLD = 0.3
with open('/app/data/black_face_ids.csv') as f:
    black_face_ids = {}
    for i, line in enumerate(f):
        line = line.strip()
        if line == '' or i == 0:
            continue
        a, b = line.split(',')
        black_face_ids[int(a)] = float(b)

# Genders & Race

In [ ]:
face_genders = get_face_genders()
face_genders = face_genders.where(
    (face_genders.labeler_id == Labeler.objects.get(name='knn-gender').id) &
    (face_genders.height >= 0.2) &
    (face_genders.in_commercial == False)
)
face_genders = face_genders.withColumn(
    'is_host2', face_genders.host_probability > 0.5)
face_genders = face_genders.withColumn(
    'start_time', face_genders.min_frame / face_genders.fps)

In [ ]:
video_to_fg = {}
for fg in face_genders.select(
    'video_id', 'face_id', 'gender_id', 'start_time', 'is_host2'
).sort('video_id', 'start_time').collect():
    video_id = fg.video_id
    if video_id not in video_to_fg:
        video_to_fg[video_id] = []
    is_black = 1 if black_face_ids.get(fg.face_id, 0) > BLACK_SCORE_THRESHOLD else 0
    is_host = 1 if fg.is_host2 else 0
    video_to_fg[video_id].append(
        (fg.gender_id, fg.start_time, is_black, is_host)
    )

print('Writing:', GENDERS_AND_RACE_PATH)
with open(GENDERS_AND_RACE_PATH, 'w') as f:
    json.dump(video_to_fg, f)
print('Done!')

# Identities

In [ ]:
face_idents = get_face_identities()
face_idents = face_idents.where(
    (face_idents.height >= 0.2) &
    (face_idents.in_commercial == False)
)
face_idents = face_idents.withColumn(
    'start_time', face_idents.min_frame / face_idents.fps)
face_idents = face_idents.withColumn(
    'is_host2', face_idents.host_probability > 0.5)

In [ ]:
video_to_fi = {}
for fi in face_idents.select(
    'video_id', 'identity_id', 'start_time', 'is_host2'
).sort('video_id', 'start_time').collect():
    video_id = fi.video_id
    if video_id not in video_to_fi:
        video_to_fi[video_id] = []
    is_host = 1 if fi.is_host2 else 0
    video_to_fi[video_id].append(
        (fi.identity_id, fi.start_time, is_host)
    )
    
print('Writing:', IDENTS_PATH)
with open(IDENTS_PATH, 'w') as f:
    json.dump(video_to_fi, f)
print('Done!')